<a href="https://colab.research.google.com/github/amitvankudre/Coursera_Capstone/blob/main/Toronto_Neighbourhoods_Week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Toronto Neighbourhoods Clustering**




## **Importing Toronto Neighbourhood Data**



In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib as plt

In [3]:
#Import eh postal codes of Canada

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(url)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
# remove the rows for which the boroughs are not assigned
df1 = df[df['Borough'] != 'Not assigned'].reindex()
df1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
# print the values for the neighbourhoods to check for any unusual / non-required values.
print('The D.F has {} rows that do not have a neighboourhood assigned'.format(df1[df1.Neighbourhood == 'Not assigned'].shape[0]))

df1.Neighbourhood.values

The D.F has 0 rows that do not have a neighboourhood assigned


array(['Parkwoods', 'Victoria Village', 'Regent Park, Harbourfront',
       'Lawrence Manor, Lawrence Heights',
       "Queen's Park, Ontario Provincial Government",
       'Islington Avenue, Humber Valley Village', 'Malvern, Rouge',
       'Don Mills', 'Parkview Hill, Woodbine Gardens',
       'Garden District, Ryerson', 'Glencairn',
       'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
       'Rouge Hill, Port Union, Highland Creek', 'Don Mills',
       'Woodbine Heights', 'St. James Town', 'Humewood-Cedarvale',
       'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood',
       'Guildwood, Morningside, West Hill', 'The Beaches', 'Berczy Park',
       'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street',
       'Christie', 'Cedarbrae', 'Hillcrest Village',
       'Bathurst Manor, Wilson Heights, Downsview North',
       'Thorncliffe Park', 'Richmond, Adelaide, King',
       'Dufferin, Dovercourt Village', 'Scarborough Village',
      

In [6]:
# get the shape of the D.F
df1.shape

(103, 3)

## **Fetching Location Data for Postal Codes**

In [7]:
# Import the geo coordinates data using csv provided
from google.colab import files
uploaded = files.upload()

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates.csv


In [8]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['Geospatial_Coordinates.csv']))
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# merge the geo data with the original D.F
df1 = df1.reindex(df2.index)
df3 = pd.merge(df1,df2,on=['Postal Code'])
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## **Foursquare Data**

In [10]:
# Foursquare credentials

CLIENT_ID = 'CEUT35WOI12KWE3CRSEZBIP2KF12AJXGN4VFMXA5JXAD11E0' # your Foursquare ID
CLIENT_SECRET = 'G1HWIVW03GEMSN0DSGJZCKQP3YTTCHCPPZ5RTCZIUX1QKNRQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CEUT35WOI12KWE3CRSEZBIP2KF12AJXGN4VFMXA5JXAD11E0
CLIENT_SECRET:G1HWIVW03GEMSN0DSGJZCKQP3YTTCHCPPZ5RTCZIUX1QKNRQ


In [11]:
# Only venues in Downtown Toronto will be explored.

df_downtown_toronto = df3[df3['Borough'] == 'Downtown Toronto'] # D.F to extract DOwntown Toronto data
df_downtown_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [12]:
#Same analysis as New York data has been performed below, for top 100 venues in Downtown Toronto

#Function to get nearby venues for each neighbourhood in Downtown Toronto
def getNearbyVenues(borough_names,neighbourhood_names, latitudes, longitudes):
  LIMIT = 100 # limit of number of venues returned by Foursquare API
  radius = 500 # define radius

  venues_list = []

  for borough,neighbourhood, lat, lng in zip(borough_names,neighbourhood_names,latitudes,longitudes) :
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)

    results = requests.get(url).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    venues_list.append([(
            borough, 
            neighbourhood,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
  
  nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
  nearby_venues.columns = ['Borough',
                  'Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
  return(nearby_venues)







In [13]:
# invoke the function to get the top venues for all the neighbourhoods in Downtown Toronto
downtown_toronto_venues = getNearbyVenues(
    df_downtown_toronto['Borough'],
    df_downtown_toronto['Neighbourhood'],
    df_downtown_toronto['Latitude'],
    df_downtown_toronto['Longitude'],
)

In [14]:
downtown_toronto_venues.head()

,Borough,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [15]:
# get the count of venues by neighbourhoods
downtown_toronto_venues.groupby('Neighbourhood').count()

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,
Berczy Park,58,58,58,58,58,58,58
Central Bay Street,60,60,60,60,60,60,60
Christie,15,15,15,15,15,15,15
"Commerce Court, Victoria Hotel",100,100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100,100
"Queen's Park, Ontario Provincial Government",37,37,37,37,37,37,37
"Regent Park, Harbourfront",45,45,45,45,45,45,45
"Richmond, Adelaide, King",96,96,96,96,96,96,96


## **One Hot Encoding**

In [16]:
# one-hot analysis
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues['Venue Category'],prefix ='',prefix_sep='')
downtown_toronto_onehot.head()

,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Rec Center,...,Pizza Place,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
downtown_toronto_onehot['Neighbourhood'] = downtown_toronto_venues['Neighbourhood']
fixed_columns = ['Neighbourhood'] + list(downtown_toronto_onehot.loc[:, downtown_toronto_onehot.columns != 'Neighbourhood'])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]
downtown_toronto_onehot.head()

,Neighbourhood,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,...,Pizza Place,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
#get the shape of the one-hot D.F
downtown_toronto_onehot.shape

(790, 163)

In [19]:
# group the one-hot D.F by mean
downtown_toronto_onehot_grouped = downtown_toronto_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_toronto_onehot_grouped.head()

,Neighbourhood,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,...,Pizza Place,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.00,0.0,0.0,0.017241,0.000000,0.00,0.0,0.000000,0.017241,0.034483,0.00,0.00,0.0,0.017241,0.017241,0.034483,0.0,0.0,0.017241,0.00,0.0,0.017241,0.0,0.000000,0.0,0.000000,0.00,0.017241,0.017241,0.0,0.000000,0.034483,0.00,0.0,0.017241,0.051724,0.086207,0.0,0.0,...,0.00,0.00,0.00,0.000000,0.00,0.017241,0.000000,0.034483,0.0,0.000000,0.00,0.00,0.0,0.000000,0.034483,0.00,0.017241,0.0,0.00,0.0,0.0,0.000000,0.0,0.017241,0.0,0.017241,0.0,0.017241,0.0,0.017241,0.00,0.00,0.017241,0.00,0.0,0.017241,0.00,0.00,0.000000,0.000000
1,Central Bay Street,0.00,0.0,0.0,0.000000,0.016667,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.033333,0.0,0.033333,0.00,0.000000,0.050000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.200000,0.0,0.0,...,0.00,0.00,0.00,0.016667,0.00,0.000000,0.016667,0.000000,0.0,0.033333,0.00,0.05,0.0,0.016667,0.016667,0.00,0.000000,0.0,0.00,0.0,0.0,0.016667,0.0,0.000000,0.0,0.000000,0.0,0.016667,0.0,0.000000,0.00,0.00,0.033333,0.00,0.0,0.016667,0.00,0.00,0.016667,0.016667
2,Christie,0.00,0.0,0.0,0.000000,0.000000,0.00,0.0,0.066667,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.200000,0.0,0.066667,0.000000,0.00,0.0,0.000000,0.000000,0.066667,0.0,0.0,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.066667,0.0,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000
3,"Commerce Court, Victoria Hotel",0.04,0.0,0.0,0.010000,0.000000,0.01,0.0,0.000000,0.000000,0.010000,0.00,0.01,0.0,0.000000,0.000000,0.020000,0.0,0.0,0.000000,0.01,0.0,0.010000,0.0,0.000000,0.0,0.010000,0.00,0.000000,0.060000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.020000,0.120000,0.0,0.0,...,0.02,0.00,0.01,0.000000,0.00,0.010000,0.000000,0.070000,0.0,0.010000,0.01,0.01,0.0,0.000000,0.030000,0.00,0.010000,0.0,0.00,0.0,0.0,0.000000,0.0,0.010000,0.0,0.010000,0.0,0.000000,0.0,0.010000,0.00,0.01,0.020000,0.00,0.0,0.020000,0.00,0.00,0.010000,0.000000
4,"Garden District, Ryerson",0.00,0.0,0.0,0.010000,0.000000,0.00,0.0,0.000000,0.000000,0.010000,0.01,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.02,0.0,0.000000,0.0,0.030000,0.0,0.010000,0.01,0.000000,0.030000,0.0,0.000000,0.000000,0.01,0.0,0.090000,0.000000,0.090000,0.0,0.0,...,0.02,0.01,0.00,0.000000,0.01,0.000000,0.020000,0.010000,0.0,0.000000,0.00,0.01,0.0,0.000000,0.010000,0.01,0.010000,0.0,0.01,0.0,0.0,0.010000,0.0,0.010000,0.0,0.010000,0.0,0.000000,0.0,0.000000,0.01,0.01,0.010000,0.02,0.0,0.000000,0.01,0.01,0.010000,0.000000


In [20]:
# get the shape of the grouped D.F
downtown_toronto_onehot_grouped.shape

(11, 163)

## **Extracting the most common venues for each neighourhood**

In [31]:
# print the top 5 venues
num_top_venues = 5

for hood in downtown_toronto_onehot_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_onehot_grouped[downtown_toronto_onehot_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2        Beer Bar  0.03
3     Cheese Shop  0.03
4  Farmers Market  0.03


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1  Italian Restaurant  0.05
2                Café  0.05
3      Sandwich Place  0.05
4         Salad Place  0.03


----Christie----
           venue  freq
0  Grocery Store  0.27
1           Café  0.20
2           Park  0.13
3    Coffee Shop  0.07
4     Baby Store  0.07


----Commerce Court, Victoria Hotel----
                 venue  freq
0          Coffee Shop  0.12
1           Restaurant  0.07
2                 Café  0.06
3                Hotel  0.06
4  American Restaurant  0.04


----Garden District, Ryerson----
                 venue  freq
0          Coffee Shop  0.09
1       Clothing Store  0.09
2       Cosmetics Shop  0.03
3  Japanese Restaurant  0.03
4                 Café  0.03


----Harbourfront East, Union Station, Toronto Islands---

In [32]:
# function to return the most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [33]:
# Create data frame showcasing the 10 most common venues for each neighbourhood
num_top_venues = 10

suffixes = ['st','nd','rd','th']

columns = ['Neighbourhood']

for i in np.arange(10) :
  if i< 3 :
      columns.append('{}{} most common venue'.format(i+1,suffixes[i]))
  else :
    columns.append('{}{} most common venue'.format(i+1,suffixes[3]))

downtown_toronto_venues_10 = pd.DataFrame(columns=columns)
downtown_toronto_venues_10.Neighbourhood = downtown_toronto_onehot_grouped.Neighbourhood


for ind in np.arange(downtown_toronto_onehot_grouped.shape[0]):
    downtown_toronto_venues_10.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_onehot_grouped.iloc[ind, :], num_top_venues)
     
downtown_toronto_venues_10.head()




,Neighbourhood,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Creperie,Fish Market
1,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Salad Place,Yoga Studio,Middle Eastern Restaurant
2,Christie,Grocery Store,Café,Park,Restaurant,Nightclub,Coffee Shop,Baby Store,Candy Store,Italian Restaurant,Yoga Studio
3,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Deli / Bodega
4,"Garden District, Ryerson",Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Hotel,Café,Italian Restaurant,Ramen Restaurant


## **Clustering the neighbourhoods**

In [25]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = downtown_toronto_onehot_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 2, 1, 0, 4, 0, 2, 3, 0, 4], dtype=int32)

In [26]:
# add clustering labels

downtown_toronto_venues_10.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_toronto_merged = df_downtown_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(downtown_toronto_venues_10.set_index('Neighbourhood'), on='Neighbourhood')

downtown_toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Pub,Park,Bakery,Café,Theater,Breakfast Spot,Electronics Store,Performing Arts Venue,Italian Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,College Cafeteria,Sushi Restaurant,Yoga Studio,Portuguese Restaurant,Japanese Restaurant,Sandwich Place,Restaurant,Burrito Place,Distribution Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Hotel,Café,Italian Restaurant,Ramen Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Creperie,Cheese Shop,Moroccan Restaurant,Cosmetics Shop,Lingerie Store
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Farmers Market,Creperie,Fish Market


In [27]:
# get the location coordinates of Toronto
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [30]:
# create map showcasing the clusters
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighbourhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters